# VM Setup

In [ ]:
AZURE_SUBSCRIPTION_ID = "add-your-subs-id"
RG_NAME = "add-your-resource-group-name"
LOCATION = "add-your-resource-group-location"

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [ ]:
!az account show

In [ ]:
VM_NAME = "add-your-vm-name"

In [ ]:
!az group create --name {RG_NAME} --location {LOCATION}

In [ ]:
VM_PARAMS = """{{
  "$schema": "https://schema.management.azure.com/schemas/2015-01-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {{
     "adminUsername": {{ "value" : "jiata"}},
     "adminPassword": {{ "value" : "Mlads99"}},
     "vmName": {{ "value" : "{}"}},
     "vmSize": {{ "value" : "Standard_NC6s_v3"}}
  }}
}}""".format(VM_NAME)

In [ ]:
!az group deployment create --resource-group {RG_NAME} --template-uri https://raw.githubusercontent.com/Azure/DataScienceVM/master/Scripts/CreateDSVM/Ubuntu/azuredeploy.json --parameters '{VM_PARAMS}'

In [ ]:
!az vm show -d -g {RG_NAME} -n {VM_NAME} --query publicIps -o table

In [ ]:
protectedSettings="""{
    "fileUris": ["https://raw.githubusercontent.com/loomlike/loomlike.github.io/master/vm_setup.sh"],
    "commandToExecute": "./vm_setup.sh"
}"""

In [ ]:
!az vm extension set --resource-group {RG_NAME} --vm-name {VM_NAME} --name customScript --publisher Microsoft.Azure.Extensions --force-update --protected-settings '{protectedSettings}'